### Study how the initial position of spreader affects the percolation and make function to keep track of how fast percolation is achieved

In [ ]:
from ca_lecture_hall_model import Status, Grid
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
# Study on initial spreader position change
def simulate_initial_spreader(
    size, density, spread_threshold, steps, initial_positions
):
    results = {}

    for position in tqdm(
        initial_positions, desc="Simulating initial spreader positions"
    ):
        grid = Grid(size, density, spread_threshold)
        grid.initialize_board()
        row, col = position
        grid.lecture_hall[row][col].set_status(Status.GOSSIP_SPREADER)

        for step in range(steps):
            grid.update_grid()
            if grid.check_percolation():
                results[position] = step + 1
                break
        else:
            results[position] = -1

    return results

In [ ]:
def plot_spreader_effect(results):
    positions = list(results.keys())
    steps = list(results.values())

    x = [pos[1] for pos in positions]
    y = [pos[0] for pos in positions]

    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(x, y, c=steps, cmap="viridis", edgecolor="k", s=100)
    plt.colorbar(scatter, label="Steps to Percolation (-1 means no percolation)")
    plt.xlabel("Column")
    plt.ylabel("Row")
    plt.title("Effect of Initial Spreader Position on Percolation Speed")
    plt.grid(True)
    plt.show()

In [ ]:
size = 50
density = 0.7
spread_threshold = 0.1
steps = 200

initial_positions = [(i, j) for i in range(size) for j in range(size)]

results = simulate_initial_spreader(
    size, density, spread_threshold, steps, initial_positions
)

plot_spreader_effect(results)

In [ ]:
def plot_distance_vs_steps(results, size):
    center = (size // 2, size // 2)
    distances = []
    steps = []

    for (row, col), step_count in results.items():
        distance_to_center = np.sqrt((row - center[0]) ** 2 + (col - center[1]) ** 2)
        distances.append(distance_to_center)
        steps.append(step_count)

    plt.plot(distances, steps, "o-", label="Steps to Percolation")
    plt.xlabel("Distance from Center")
    plt.ylabel("Steps to Percolation")
    plt.title("Effect of Distance from Center on Percolation Steps")
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
plot_distance_vs_steps(results, size=50)

In [ ]:
def analyze_center_vs_edge(results, size):
    center = (size // 2, size // 2)
    center_radius = size // 4

    center_steps = []
    edge_steps = []

    for (row, col), step_count in results.items():
        distance_to_center = np.sqrt((row - center[0]) ** 2 + (col - center[1]) ** 2)
        if distance_to_center <= center_radius:
            center_steps.append(step_count)
        else:
            edge_steps.append(step_count)

    avg_center = np.mean(center_steps)
    avg_edge = np.mean(edge_steps)

    print(f"Average steps to percolation (center): {avg_center}")
    print(f"Average steps to percolation (edge): {avg_edge}")

    plt.boxplot([center_steps, edge_steps], labels=["Center", "Edge"])
    plt.ylabel("Steps to Percolation")
    plt.title("Comparison of Percolation Steps (Center vs Edge)")
    plt.grid(True)
    plt.show()


analyze_center_vs_edge(results, size=50)

In [ ]:
def plot_distance_vs_percolation_heatmap(results, size):
    center = (size // 2, size // 2)
    distance_map = {}

    for (row, col), steps in results.items():
        distance = np.sqrt((row - center[0]) ** 2 + (col - center[1]) ** 2)
        if steps != -1:
            distance_map.setdefault(distance, []).append(steps)

    avg_steps = {d: np.mean(steps) for d, steps in distance_map.items()}

    distances = list(avg_steps.keys())
    percolation_steps = list(avg_steps.values())

    plt.scatter(distances, percolation_steps, c=percolation_steps, cmap="viridis")
    plt.colorbar(label="Average Steps to Percolation")
    plt.xlabel("Distance from Center")
    plt.ylabel("Average Steps to Percolation")
    plt.title("Distance from Center vs Average Percolation Steps")
    plt.grid(True)
    plt.show()


plot_distance_vs_percolation_heatmap(results, size=50)

In [ ]:
size = 50
density = 0.3
spread_threshold = 0.1
steps = 100

initial_positions = [(i, j) for i in range(size) for j in range(size)]

results = simulate_initial_spreader(
    size, density, spread_threshold, steps, initial_positions
)

plot_spreader_effect(results)

In [ ]:
# Study on initial spreader position change with spread intention
def simulate_initial_spreader_with_intention(
    size, density, spread_threshold, steps, initial_positions, center_boost=0.2
):
    results = {}

    # Define the central region
    center_start = size // 4
    center_end = 3 * (size // 4)

    for position in tqdm(
        initial_positions, desc="Simulating initial spreader positions"
    ):
        grid = Grid(size, density, spread_threshold)
        grid.initialize_board()
        row, col = position

        # Check if the position is in the central region
        if center_start <= row < center_end and center_start <= col < center_end:
            spreading_prob = min(
                1.0, spread_threshold + center_boost
            )  # Increase spread intention in the center
        else:
            spreading_prob = spread_threshold  # Normal spread intention in the edges

        # Set the initial spreader's status and spreading probability
        grid.lecture_hall[row][col].set_status(Status.GOSSIP_SPREADER)
        grid.lecture_hall[row][col].set_spreading_prob(spreading_prob)

        # Simulate the spread
        for step in range(steps):
            grid.update_grid()
            if grid.check_percolation():
                results[position] = step + 1
                break
        else:
            results[position] = -1

    return results


size = 50
density = 0.7
spread_threshold = 0.1
steps = 100
center_boost = 0.7

initial_positions = [(i, j) for i in range(size) for j in range(size)]

results = simulate_initial_spreader_with_intention(
    size, density, spread_threshold, steps, initial_positions, center_boost
)

# Plot the results (reuse the plotting function for visualizing percolation speed)
plot_spreader_effect(results)